In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
warnings.filterwarnings("ignore", category=UserWarning)
import pickle
import shap
import common_f as cf

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, \
                                    StratifiedKFold
from sklearn.metrics import confusion_matrix,precision_recall_curve, roc_auc_score, \
                            roc_curve, auc, confusion_matrix, \
                            accuracy_score, f1_score, recall_score, \
                            precision_score, balanced_accuracy_score, \
                            confusion_matrix

random_seed = 923

current_path = os.getcwd()
X_train_filePath = current_path + '/Data/X_train.csv'
y_train_filePath = current_path + '/Data/y_train.csv'
X_test_filePath = current_path + '/Data/X_test.csv'
y_test_filePath = current_path + '/Data/y_test.csv'
feature_filePath = current_path + '/Data/selected_features.csv'

X_train_val = pd.read_csv(X_train_filePath, encoding = 'utf-8')
y_train_val = pd.read_csv(y_train_filePath, encoding = 'utf-8')
y_train_val = y_train_val.values.ravel()

X_test = pd.read_csv(X_test_filePath, encoding = 'utf-8')
y_test = pd.read_csv(y_test_filePath, encoding = 'utf-8')
y_test = y_test.values.ravel()

feature_df = pd.read_csv(feature_filePath, encoding = 'utf-8')
SELECTED_FEATURES = [i for i in feature_df['selected Features']]

X_selctd_train_val = X_train_val[SELECTED_FEATURES]
X_selctd_test = X_test[SELECTED_FEATURES]
print(SELECTED_FEATURES)

skFold = StratifiedKFold(n_splits = 5, random_state = random_seed,
                         shuffle = True)

scoring = 'accuracy'

In [2]:
logreg_pipeline = Pipeline(steps = [('scale', StandardScaler()),
                                    ('LR', LogisticRegression(random_state = random_seed, 
                                                              max_iter = 20000))])

rf_pipeline = Pipeline(steps = [('scale',StandardScaler()),
                                ('RF', RandomForestClassifier(random_state = random_seed))])

xgb_pipeline = Pipeline(steps = [('scaler', StandardScaler()),
                                 ('XGB', XGBClassifier(random_state = random_seed))])

lgbm_pipeline = Pipeline(steps = [('scaler', StandardScaler()),
                                  ('LGBM', LGBMClassifier(random_state = random_seed))])

svm_pipeline = Pipeline(steps = [('scaler', StandardScaler()),
                                 ('SVC', SVC(probability = True, 
                                             random_state = random_seed))])

lr_best_params_path = current_path + \
                      f'/Results/best_params/best_lr_params_{random_seed}.pkl'
rf_best_params_path = current_path + \
                      f'/Results/best_params/best_rf_params_{random_seed}.pkl'
xgb_best_params_path = current_path + \
                       f'/Results/best_params/best_xgb_params_{random_seed}.pkl'
lgbm_best_params_path = current_path + \
                        f'/Results/best_params/best_lgbm_params_{random_seed}.pkl'
svm_best_params_path = current_path + \
                       f'/Results/best_params/best_svm_params_{random_seed}.pkl'

In [ ]:
# /********** Logistic Regression hyperparameter tuning **********/

# NOTE:Fine-tuning LR hyperparameters here
lr_para_grid = {
    'LR__C': [0.005, 0.01, 0.05, 0.06, 0.1, 0.5, 1, 10, 100],
    'LR__penalty': ['l1', 'l2', 'elasticnet', None],
    'LR__solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg', 'sag'],
    'LR__dual': [True, False],
    'LR__fit_intercept': [True, False]
}

lr_random_search = RandomizedSearchCV(logreg_pipeline, n_iter = 50000,
                                      param_distributions = lr_para_grid,
                                      cv = skFold, scoring = scoring,
                                      random_state = random_seed, n_jobs = -1)
lr_random_search.fit(X_selctd_train_val, y_train_val)
best_lr_params = lr_random_search.best_params_

with open(lr_best_params_path, 'wb') as f:
    pickle.dump(best_lr_params, f)

In [ ]:
# /********** Random Forest hyperparameter tuning **********/

# NOTE:Fine-tuning RF hyperparameters here
rf_para_grid = {
    'RF__n_estimators': [i for i in range(100, 1100, 100)],
    'RF__max_features': ['sqrt', 'log2', None],
    'RF__max_depth': [None, 3, 5, 7, 9, 10, 11, 12, 13, 15],
    'RF__min_samples_split': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 13, 15, 50, 100],
    'RF__criterion': ['gini', 'entropy'],
    'RF__min_samples_leaf': [3, 5, 7, 9, 11, 13, 15, 20, 30, 40, 50, 100]
}

rf_random_search = RandomizedSearchCV(rf_pipeline, n_iter = 50000,
                                      param_distributions = rf_para_grid,
                                      cv = skFold, scoring = scoring,
                                      random_state = random_seed, n_jobs = -1)
rf_random_search.fit(X_selctd_train_val, y_train_val)
best_rf_params = rf_random_search.best_params_

with open(rf_best_params_path, 'wb') as f:
    pickle.dump(best_rf_params, f)

In [ ]:
# /********** XGBoost hyperparameter tuning **********/

# NOTE:Fine-tuning XGB hyperparameters here
xgb_param_grid = {
    'XGB__learning_rate': [0.3, 0.1, 0.01, 0.001],
    'XGB__n_estimators': [i for i in range(100, 1100, 100)],
    'XGB__max_depth': [i for i in range(3, 18, 3)],
    'XGB__min_child_weight': [i for i in range(3, 18, 3)],
    'XGB__gamma': [i/10.0 for i in range(0,20)],
    'XGB__colsample_bytree': [i/100.0 for i in range(75, 105, 5)],
    'XGB__subsample': [i/100.0 for i in range(70, 105, 5)],
    'XGB__reg_alpha': [1e-5, 1e-2, 0.1, 0, 1, 100],
    'XGB__eta': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
}

xgb_random_search = RandomizedSearchCV(xgb_pipeline, n_iter = 50000,
                                       param_distributions = xgb_param_grid,
                                       cv = skFold, scoring = scoring,
                                       random_state = random_seed, n_jobs = -1)
xgb_random_search.fit(X_selctd_train_val, y_train_val)
best_xgb_params = xgb_random_search.best_params_

with open(xgb_best_params_path, 'wb') as f:
    pickle.dump(best_xgb_params, f)

In [ ]:
# /********** LightGBM hyperparameter tuning **********/

# NOTE:Fine-tuning LightGBM hyperparameters here
lgbm_param_grid = {
    'LGBM__learning_rate': [0.1, 0.01, 0.001, 0.0001, 0.00001],
    'LGBM__n_estimators': [i for i in range(100, 1100, 100)],
    'LGBM__max_depth': [i for i in range(3, 33, 3)],
    'LGBM__num_leaves': [i for i in range(10, 110, 10)],
    'LGBM__subsample': [i/100.0 for i in range(70, 105, 5)],
    'LGBM__colsample_bytree': [i/100.0 for i in range(70, 105, 5)],
    'LGBM__min_child_samples': [10, 13, 15, 16, 18, 19, 20, 21, 22],
    'LGBM__min_child_weight': [0.001, 0.002],
    'LGBM__feature_fraction': [0.6, 0.8, 1],
    'LGBM__bagging_fraction': [0.8, 0.9, 1],
    'LGBM__bagging_freq': [0, 1, 2, 3, 4],
    'LGBM__cat_smooth': [0, 10, 20]
}

lgbm_random_search = RandomizedSearchCV(lgbm_pipeline, n_iter = 50000,
                                        param_distributions = lgbm_param_grid,
                                        cv = skFold, scoring = scoring,
                                        random_state = random_seed, n_jobs = -1)
lgbm_random_search.fit(X_selctd_train_val, y_train_val)
best_lgbm_params = lgbm_random_search.best_params_

with open(lgbm_best_params_path, 'wb') as f:
    pickle.dump(best_lgbm_params, f)

In [ ]:
# /********** SVM hyperparameter tuning **********/

# NOTE:Fine-tuning SVM hyperparameters here
svm_param_grid = {
    'SVC__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'SVC__kernel': ['linear', 'rbf', 'poly', 'sigmoid', 'precomputed'],
    'SVC__gamma': ['scale', 'auto', 0.0001, 0.001, 0.01, 0.1]
}

svm_random_search = RandomizedSearchCV(svm_pipeline, n_iter = 50000,
                                        param_distributions = svm_param_grid,
                                        cv = skFold, scoring = scoring,
                                        random_state = random_seed, n_jobs = -1)
svm_random_search.fit(X_selctd_train_val, y_train_val)
best_svm_params = svm_random_search.best_params_

with open(svm_best_params_path, 'wb') as f:
    pickle.dump(best_svm_params, f)

In [ ]:
with open(lr_best_params_path, 'rb') as f:
    loaded_best_lr_params = pickle.load(f)
print('loaded_best_lr_params:\n', loaded_best_lr_params)

with open(svm_best_params_path, 'rb') as f:
    loaded_best_svm_params = pickle.load(f)
print('loaded_best_svm_params:\n', loaded_best_svm_params)

with open(rf_best_params_path, 'rb') as f:
    loaded_best_rf_params = pickle.load(f)
print('loaded_best_rf_params:\n', loaded_best_rf_params)

with open(xgb_best_params_path, 'rb') as f:
    loaded_best_xgb_params = pickle.load(f)
print('loaded_best_xgb_params:\n', loaded_best_xgb_params)

with open(lgbm_best_params_path, 'rb') as f:
    loaded_best_lgbm_params = pickle.load(f)
print('loaded_best_lgbm_params:\n', loaded_best_lgbm_params)

print('best lr params is:', loaded_best_lr_params)
print('best svm params is:', loaded_best_svm_params)
print('best rf params is:', loaded_best_rf_params)
print('best xgb params is:', loaded_best_xgb_params)
print('best lgbm params is:', loaded_best_lgbm_params)

In [ ]:
logreg_pipeline.set_params(**loaded_best_lr_params)
svm_pipeline.set_params(**loaded_best_svm_params)
rf_pipeline.set_params(**loaded_best_rf_params)
xgb_pipeline.set_params(**loaded_best_xgb_params)
lgbm_pipeline.set_params(**loaded_best_lgbm_params)

pipelines = [
   ('LR', logreg_pipeline),
   ('SVM', svm_pipeline),
   ('RF', rf_pipeline),
   ('XGBoost', xgb_pipeline),
   ('LightGBM', lgbm_pipeline)
   ]

results_all_pipelines = {name: [] for name, _ in pipelines}
pred_prob_list = {name: [] for name, _ in pipelines}
val_true_list = []

for train_idx, val_idx in skFold.split(X_selctd_train_val, y_train_val):
    y = pd.Series(y_train_val)
    X_train, X_val = X_selctd_train_val.iloc[train_idx,:], \
                     X_selctd_train_val.iloc[val_idx,:]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    for name, pipeline in pipelines:
      pipeline.fit(X_train, y_train)
      y_val_pred = pipeline.predict(X_val)
      y_val_proba = pipeline.predict_proba(X_val)[:, 1]
      metrics = cf.evaluate_model(y_val, y_val_pred, y_val_proba)
      results_all_pipelines[name].append(metrics)
      pred_prob_list[name].append(np.array(y_val_proba))

    val_true_list.append(np.array(y_val))

probs_paths = []
for model_name, prob_lists in pred_prob_list.items():
    df = pd.DataFrame(prob_lists).T
    df.columns = [f'Fold_{i+1}' for i in range(len(prob_lists))]
    prob_filename = f'/Data/{model_name}_pred_prob.csv'
    probs_paths.append(current_path + prob_filename)
    df.to_csv(current_path + prob_filename, index = False)

val_list_df = pd.DataFrame(val_true_list).T
val_list_df.columns = [f'Fold_{i+1}' for i in range(len(val_true_list))]
var_filename = '/Data/cv_val_true_value.csv'
val_list_df.to_csv(current_path + var_filename, index = False)

labels = ['LR            ',
          'SVM         ',
          'RF            ',
          'XGBoost   ',
          'LightGBM '
          ]

colors = [['midnightblue', 'lightskyblue'],
          ['mediumblue', 'mediumslateblue'],
          ['firebrick', 'salmon'], 
          ['darkgreen', 'lime'],
          ['orangered', 'lightsalmon']
          ]

results_all_pipelines_df = {name: pd.DataFrame(metrics_list) \
                            for name, metrics_list in results_all_pipelines.items()}

cv_val_avg_results = {}
for pipeline_name, df in results_all_pipelines_df.items():
    mean_results = df.mean()
    std_results = df.std()
    cv_val_avg_results[pipeline_name] = {f'{metric}_mean' : mean_results[metric] \
                                for metric in mean_results.index}
    cv_val_avg_results[pipeline_name].update({f'{metric}_std' : std_results[metric] \
                                     for metric in std_results.index})

cv_val_avg_results_df = pd.DataFrame(cv_val_avg_results).T
cv_val_avg_results_df.to_csv(current_path + '/Results/cv_val_avg_results.csv',
                             encoding = 'utf-8', index = True)

roc_avg_list = cv_val_avg_results_df[['ROC AUC_mean', 'ROC AUC_std']]
roc_avg_list.reset_index(drop=True, inplace=True)
cv_avg_roc_fileName = 'cv_avg_roc_curve'
cf.plot_multi_roc_curves(probs_paths, current_path + var_filename, 
                         labels, colors, roc_avg_list, cv_avg_roc_fileName)
print(cv_val_avg_results_df, cv_val_avg_results_df['ROC AUC_mean'],
      cv_val_avg_results_df['ROC AUC_std'])

In [ ]:
logreg_pipeline.fit(X_selctd_train_val, y_train_val)
svm_pipeline.fit(X_selctd_train_val, y_train_val)
rf_pipeline.fit(X_selctd_train_val, y_train_val)
xgb_pipeline.fit(X_selctd_train_val, y_train_val)
lgbm_pipeline.fit(X_selctd_train_val, y_train_val)

logreg_pipeline.set_params(**loaded_best_lr_params)
svm_pipeline.set_params(**loaded_best_svm_params)
rf_pipeline.set_params(**loaded_best_rf_params)
xgb_pipeline.set_params(**loaded_best_xgb_params)
lgbm_pipeline.set_params(**loaded_best_lgbm_params)

In [ ]:
## Plot shap
explainer = shap.Explainer(lgbm_pipeline.predict, X_selctd_test)
shap_values = explainer(X_selctd_test)
print(X_selctd_test.columns)
shap_fileName = 'lgbm_shap'
cf.plot_shap(explainer, shap_values, X_selctd_test, fileName=shap_fileName)

feature_names = X_selctd_test.columns.tolist()
feature_imp = np.abs(shap_values.values).mean(axis = 0)
total_imp = dict(zip(feature_names, feature_imp.tolist()))
imp_df = pd.DataFrame({'Features': list(total_imp.keys()),
                          'imp_cv': list(total_imp.values())})

imp_df.sort_values(by = 'imp_cv', ascending = False, inplace = True)
cf.plot_importances(imp_df['imp_cv'], imp_df['Features'], 'LightGBM')

imp_df.columns = ['Features', 'Importances']
imp_df.to_csv(current_path + '/Data/feature_imp.csv')

In [ ]:
# Make dataframes to plot

lr_pred = logreg_pipeline.predict(X_selctd_test)
rf_pred = rf_pipeline.predict(X_selctd_test)
xgb_pred = xgb_pipeline.predict(X_selctd_test)
lgbm_pred = lgbm_pipeline.predict(X_selctd_test)
svm_pred = svm_pipeline.predict(X_selctd_test)

lr_sen_spe = cf.calculate_sensitivity_specificity(y_test, lr_pred)
lr_df = pd.DataFrame(data=[f1_score(y_test,lr_pred),accuracy_score(y_test, lr_pred), recall_score(y_test, lr_pred),
                   precision_score(y_test, lr_pred), roc_auc_score(y_test, lr_probs),lr_sen_spe[0], lr_sen_spe[1],
                   balanced_accuracy_score(y_test, lr_pred)], 
             columns=['LR Score'],
             index=["F1 Score","Accuracy", "Recall", "Precision", "AUC(95%)CI", "Sensitivity", "Specificity", "Balanced accuracy"])

rf_sen_spe = cf.calculate_sensitivity_specificity(y_test, rf_pred)
rf_df = pd.DataFrame(data=[f1_score(y_test,rf_pred),accuracy_score(y_test, rf_pred), recall_score(y_test, rf_pred),
                   precision_score(y_test, rf_pred), roc_auc_score(y_test, rf_probs),rf_sen_spe[0], rf_sen_spe[1],
                   balanced_accuracy_score(y_test, rf_pred)], 
             columns=['RF Score'],
             index=["F1 Score","Accuracy", "Recall", "Precision", "AUC(95%)CI", "Sensitivity", "Specificity", "Balanced accuracy"])

xgb_sen_spe = cf.calculate_sensitivity_specificity(y_test, xgb_pred)
xgb_df = pd.DataFrame(data=[f1_score(y_test,xgb_pred),accuracy_score(y_test, xgb_pred), recall_score(y_test, xgb_pred),
                   precision_score(y_test, xgb_pred), roc_auc_score(y_test, xgb_probs),xgb_sen_spe[0], xgb_sen_spe[1],
                   balanced_accuracy_score(y_test, xgb_pred)], 
             columns=['XGBoost Score'],
             index=["F1 Score","Accuracy", "Recall", "Precision", "AUC(95%)CI", "Sensitivity", "Specificity", "Balanced accuracy"])

lgbm_sen_spe = cf.calculate_sensitivity_specificity(y_test, lgbm_pred)
lgbm_df = pd.DataFrame(data=[f1_score(y_test,lgbm_pred),accuracy_score(y_test, lgbm_pred), recall_score(y_test, lgbm_pred),
                   precision_score(y_test, lgbm_pred), roc_auc_score(y_test, lgbm_probs),lgbm_sen_spe[0], lgbm_sen_spe[1],
                   balanced_accuracy_score(y_test, lgbm_pred)], 
             columns=['LightGBM Score'],
             index=["F1 Score","Accuracy", "Recall", "Precision", "AUC(95%)CI", "Sensitivity", "Specificity", "Balanced accuracy"])

svm_sen_spe = cf.calculate_sensitivity_specificity(y_test, svm_pred)
svm_df = pd.DataFrame(data=[f1_score(y_test,svm_pred),accuracy_score(y_test, svm_pred), recall_score(y_test, svm_pred),
                   precision_score(y_test, svm_pred), roc_auc_score(y_test, svm_probs),svm_sen_spe[0], svm_sen_spe[1],
                   balanced_accuracy_score(y_test, svm_pred)], 
             columns=['SVM Score'],
             index=["F1 Score","Accuracy", "Recall", "Precision", "AUC(95%)CI", "Sensitivity", "Specificity", "Balanced accuracy"])

df_models = round(pd.concat([lr_df, rf_df, xgb_df, lgbm_df, svm_df], axis= 1), 5)

colors = ["lightgray","lightgray","#0f4c81"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
background_color = "#fbfbfb"

fig = plt.figure(figsize = (30, 30), dpi = 1000) 
gs = fig.add_gridspec(9, 2)
gs.update(wspace = 0.2, hspace = 0.5)
ax0 = fig.add_subplot(gs[0, :])

ax_heatmap = sns.heatmap(df_models.T, cmap=colormap, annot = True, fmt=".1%",
                         vmin = 0, vmax = 0.95, linewidths = 2.5,cbar = False, ax= ax0,annot_kws = {"fontsize":22})
ax_heatmap.set_xticklabels(ax_heatmap.get_xticklabels(), fontsize = 22)
ax_heatmap.set_yticklabels(ax_heatmap.get_yticklabels(), fontsize = 22)
fig.patch.set_facecolor(background_color)
ax0.set_facecolor(background_color) 

ax0.text(0, -0.5,'Model Comparison', fontsize = 28, fontweight = 'bold', fontfamily = 'serif')
ax0.tick_params(axis = u'both', which = u'both', length = 0)

plt.savefig(current_path + '/Results/results.png', dpi = 1000,
            bbox_inches = 'tight', transparent = False)
plt.show()

In [ ]:
# Plotting Metrics

lr_cm = confusion_matrix(y_test, lr_pred)
rf_cm  = confusion_matrix(y_test, rf_pred)
xgb_cm  = confusion_matrix(y_test, xgb_pred)
lgbm_cm = confusion_matrix(y_test, lgbm_pred)
svm_cm  = confusion_matrix(y_test, svm_pred)

colors = ["lightgray","#0f4c81","#0f4c81","#0f4c81","#0f4c81","#0f4c81","#0f4c81","#0f4c81"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

background_color = "#fbfbfb"

fig = plt.figure(figsize = (10,14), dpi = 800)
gs = fig.add_gridspec(6, 2)
gs.update(wspace = 0.1, hspace = 0.4)
ax0 = fig.add_subplot(gs[0, :])
ax1 = fig.add_subplot(gs[1, :])
ax2 = fig.add_subplot(gs[2, :])
ax3 = fig.add_subplot(gs[3, :])
ax4 = fig.add_subplot(gs[4, :])

sns.heatmap(lr_cm, cmap = colormap,annot=True, fmt = "d", linewidths = 5, cbar = False, ax = ax0,
            yticklabels = ['Actual 0','Actual 1'], xticklabels = ['Predicted 0','Predicted 1'], annot_kws = {"fontsize":12})

sns.heatmap(rf_cm, cmap = colormap, annot = True, fmt = "d", linewidths = 5, cbar = False, ax = ax1,
                     yticklabels = ['Actual 0','Actual 1'], xticklabels = ['Predicted 0','Predicted 1'], annot_kws = {"fontsize":12})

sns.heatmap(xgb_cm, cmap = colormap, annot = True, fmt = "d", linewidths=5, cbar = False, ax = ax2,
            yticklabels = ['Actual 0','Actual 1'], xticklabels = ['Predicted 0','Predicted 1'], annot_kws = {"fontsize":12})

sns.heatmap(lgbm_cm, cmap = colormap, annot = True, fmt = "d", linewidths = 5, cbar = False, ax = ax3,
            yticklabels = ['Actual 0','Actual 1'], xticklabels = ['Predicted 0','Predicted 1'], annot_kws = {"fontsize":12})

sns.heatmap(svm_cm, cmap = colormap, annot = True, fmt = "d", linewidths = 5, cbar = False, ax = ax4,
            yticklabels = ['Actual 0','Actual 1'], xticklabels = ['Predicted 0','Predicted 1'], annot_kws = {"fontsize":12})

background_color = "#fbfbfb"
fig.patch.set_facecolor(background_color)
ax0.tick_params(axis = u'both', which = u'both', length=  0)
ax0.set_facecolor(background_color) 
ax1.tick_params(axis = u'both', which = u'both', length = 0)
ax1.set_facecolor(background_color) 
ax2.tick_params(axis = u'both', which = u'both', length = 0)
ax2.set_facecolor(background_color)
ax3.tick_params(axis = u'both', which = u'both', length = 0)
ax3.set_facecolor(background_color)
ax4.tick_params(axis = u'both', which = u'both', length = 0)
ax4.set_facecolor(background_color)

ax0.text(0, -0.2, 'LR Performance', fontsize = 12, fontweight = 'bold', fontfamily = 'serif')
ax1.text(0, -0.2, 'RF Performance', fontsize = 12, fontweight = 'bold', fontfamily = 'serif')
ax2.text(0, -0.2, 'XGBoost Performance', fontsize = 12, fontweight = 'bold', fontfamily = 'serif')
ax3.text(0, -0.2, 'LightGBM Performance', fontsize = 12, fontweight = 'bold', fontfamily = 'serif')
ax4.text(0, -0.2, 'SVM Performance', fontsize = 12, fontweight = 'bold', fontfamily = 'serif')

plt.savefig(current_path + '/Results/matrix.png', dpi = 1000,
            bbox_inches = 'tight', transparent = False)
plt.show()

In [ ]:
# /********** external validation **********/

evSet_filePath = current_path + '/Data/jinzhai_preprocessed_raw_data_23_24.csv'
csvPath = os.path.abspath(os.path.join(evSet_filePath, ".."))
EVSet = pd.read_csv(evSet_filePath, encoding = 'utf-8')
print(EVSet.columns,EVSet.shape)

columns_to_drop = ['Pulse',
                   'Lipoprotein(a) (LP(a))', 
                   'Apolipoprotein B (ApoB)',
                   'Salty or mild taste preferences',
                   'Frequency of vegetable consumption',
                   'Frequency of fruit consumption',
                   'Mental stress',
                   'Frequency of getting angry',
                   'Emotions causing physical discomfort',
                   'History of cerebrovascular disease',
                   'History of hypertension',
                   'History of diabetes',
                   'History of dyslipidemia',
                   'History of cardiac',
                   'Family history of stroke',
                   'Family history of coronary artery disease',
                   'Family history of hypertension',
                   'Family history of diabetes',
                   ]

EVSet = EVSet.rename(columns={'Body mass index (BMI)' : 'BMI',
                              'Abdominal circumference (AC)' : 'WC',
                              'Systolic blood pressure (SBP)' : 'SBP',
                              'Diastolic blood pressure (DBP)' : 'DBP',
                              'Low-density lipoprotein cholesterol (LDL-C)' : 'LDL-C',
                              'Triglyceride (TG)' : 'TG',
                              'High-density lipoprotein cholesterol (HDL-C)' : 'HDL-C',
                              'Total cholesterol (TC)' : 'TC',
                              'Non-high-density lipoprotein cholesterol (Non-HDL-C)' : 'Non-HDL-C',
                              'Fasting blood glucose (FBG)' : 'FBG',
                              'Triglyceride-glucose index (TyG Index)' : 'TyG Index',
                              'Uric acid (UA)' : 'UA',
                              'Creatinine (Cr)' : 'Cr',
                              'Estimated glomerular filtration rate (eGFR)' : 'eGFR',
                              'Homocysteine (Hcy)' : 'Hcy',
                              'Hemoglobin A1c (HbA1c)' : 'HbA1c'
                              })

multi_catog_variables = ["Carnivorous or vegetarian preferences"]

EVSet.drop(columns=columns_to_drop, inplace=True)
EVSet.to_csv(os.path.join(csvPath, 'CSVD_EV_DataSet_23_24.csv'),
            encoding = 'utf-8', index = None)

if len(multi_catog_variables) > 0:
    diet_column = 'Carnivorous or vegetarian preferences'
    if diet_column in multi_catog_variables:
        multi_catog_variables.remove(diet_column)
        diet_series = EVSet[[diet_column]]
        diet_encoder = OneHotEncoder()
        diet_encoded_data = diet_encoder.fit_transform(diet_series).toarray()
        diet_columns = np.array(['Vegetable-heavy diet', 'Balanced diet', 'Meat-heavy diet'])
        diet_encoded_df = pd.DataFrame(diet_encoded_data, columns=diet_columns)
        EVSet = EVSet.drop(columns = [diet_column])
        EVSet = pd.concat([EVSet, diet_encoded_df], axis=1)
    if len(multi_catog_variables) > 0:
        encoder = OneHotEncoder()
        multi_categ_cols = EVSet[multi_catog_variables]
        encoded_data = encoder.fit_transform(multi_categ_cols).toarray()
        encoded_df = pd.DataFrame(encoded_data,
                                columns = encoder.get_feature_names_out(multi_catog_variables))
        EVSet = EVSet.drop(columns = multi_catog_variables)
        EVSet = pd.concat([EVSet, encoded_df], axis = 1)
    EVSet.to_csv(os.path.join(csvPath, 'EV_dataSet_oneHot_23_24.csv'),
                encoding = 'utf-8', index = None)

X_EVSet = EVSet.drop('CSVD', axis = 1)
y_EVSet = EVSet['CSVD']
print(X_EVSet.columns)
X_selctd_EVSet = X_EVSet[SELECTED_FEATURES]

e_probs = lgbm_pipeline.predict_proba(X_selctd_EVSet)[:, 1]
e_pred = lgbm_pipeline.predict(X_selctd_EVSet)



sen_spe = cf.calculate_sensitivity_specificity(y_EVSet, e_pred)
result_EV_df = pd.DataFrame(data=[f1_score(y_EVSet,e_pred),
                                  accuracy_score(y_EVSet, e_pred),
                                  recall_score(y_EVSet, e_pred),
                                  precision_score(y_EVSet, e_pred),
                                  roc_auc_score(y_EVSet, e_probs),
                                  sen_spe[0],
                                  sen_spe[1],
                                  balanced_accuracy_score(y_EVSet, e_pred)],
                            columns=['best_model_performance'],
                            index=["F1 Score", "Accuracy", "Recall",
                                   "Precision", "AUC(95%)CI",
                                   "Sensitivity", "Specificity",
                                   "Balanced accuracy"])

result_EV_df.to_csv(current_path + '/Results/ev_results_23_24.csv',
                             encoding = 'utf-8', index = True)
print(result_EV_df)

e_cm  = confusion_matrix(y_EVSet, e_pred)

e_fig = plt.figure(figsize = (10,3), dpi = 800)

colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
sns.heatmap(e_cm, cmap = colormap,annot = True,
            fmt = "d",linewidths = 5, cbar = False,
            yticklabels = ['Actual 0','Actual 1'],
            xticklabels = ['Predicted 0','Predicted 1'],
            annot_kws = {"fontsize":12})
plt.savefig(current_path + '/Results/e_matrix_23_24.png', dpi = 1000,
            bbox_inches = 'tight', transparent = False)
plt.show()


best_model = lgbm_pipeline
best_params = loaded_best_lgbm_params
best_model.set_params(**best_params)

y_true_test = y_test
y_true_eVal = y_EVSet

y_probs_test = best_model.predict_proba(X_selctd_test)[:, 1]
y_probs_eVal = best_model.predict_proba(X_selctd_EVSet)[:, 1]

y_true_fina_list = []
y_true_fina_list.append(y_true_test)
y_true_fina_list.append(y_true_eVal)
y_probs_fina_list = []
y_probs_fina_list.append(y_probs_test)
y_probs_fina_list.append(y_probs_eVal)

fina_labels = ['test set', 'EV set']
fina_colors = ['#FF7F0E', '#2E5662']
test_val_roc_name = 'test_eval_roc_curve_23_24__'
test_val_pr_name = 'test_eval_pr_curve_23_24__'
cf.plot_roc_curves(y_true_fina_list, y_probs_fina_list,
                   fina_labels, fina_colors, test_val_roc_name)
cf.plot_pr_curves(y_true_fina_list, y_probs_fina_list,
                  fina_labels, fina_colors, test_val_pr_name)